### Введение

Элементарная [реализация](binary%20search%20tree.ipynb) двоичного дерева поиска позволяет выполнять необходимые операции вставки, поиска, порядковых операций в среднем за $O(logN)$, что обеспечивается случайным порядком прихода новых элементов и _двоичностью дерева_, но по мере добавления/удаления элементов дерево (особенно, если поступающие элементы отранжированы, тогда дерево вообще вырождается в список) постепенно разбалансируется, что приводит к среднему времени операций ~ $O(\sqrt{N})$. С целью поддержания логарифмической сложности операций для любого числа элементов была предложена идея __самобалансирующихся деревьев (`balanced search trees`)__, о которых и пойдет речь.

### 1. 2-3 дерево (2-3 tree)

__2-3 деревом__ называют дерево поиска, допускающее кроме существования узлов с 1 ключом и 2 потомками (собственно, _2-узлов_) также существования узлов с 2 ключами и 3 потомками (_3-узлов_). Поддержание такого дерева гарантирует, что расстояние от корня до любого терминального узла одинаково.


![2-3](support/balst "2-3-tree-structure")

Структура 2-узла идентична структуре узла обычного `BST`.

Структура 3-узла с ключами (`KEY_L`, `KEY_G`) следующая: 
- __левая__ ветвь ведет либо в `null` либо в корень поддерева, в котором __все ключи меньше `KEY_L`__; 
- __средняя__ ветвь ведет либо в `null` либо в корень поддерева, в котором __все ключи между `KEY_L` и `KEY_G`__;
- __правая__ ветвь ведет либо в `null`, либо в корень поддерева, в котором __все ключи больше `KEY_G`__.

Очевиден порядок действий для поиска по ключу `key`: если текущий узел `null`, вернем `null`, иначе сравним `key` последовательно со всеми ключами в узле (их будет 2 или 3) и отправимся в соответствующий узел.  

__Вставка элемента__ `(key:value)` __в 2-узел__ требует его превращения в 3-узел: в зависимости от соотношения между ключами новый элемент становится левым или правым в новом узле.

__Вставка элемеента__ `(key:value)` __после 3-узла__ происходит поинтереснее (здесь и происходит балансировка дерева). 
+ Текущий 3-узел `NODE` превращаем во временный 4-узел с 3 ключами и 4 ветвями;
+ Средний ключ `KEY_M` в `NODE` отправляем выше по дереву как `SUBROOT`, и оставшийся 4-узел с 2 ключами естественным образом разбиваем на два 2-узла, являющихся потомками `SUBROOT`.
+ Если нужно перестроить узел выше после преобразования, делаем аналогично.
+ Если дошли до `root`, и он стал 4-узлом, разбиваем на три 2-узла.


![2-3-insert](support/twtrtr "3-node insertion")

Нетрудно понять, что глубина дерева будет увеличиваться на 1 только в том случае, если вставка производится в 3-узел, и все вышестоящие узлы вплоть до `root` являются 3-узлами. 

Непосредственная реализация 2-3-дерева в таком виде сложна по нескольким причинам (два типа узлов, несколько случаев вставки). Удача в том, что есть изящная реализация идеи 2-3-дерева, не имеющая этих сложностей (но имеющая другие :)).

И это - 

### 2. Красно-черное дерево (Red-black tree)

Идея __красно-черного дерева__ состоит в том, чтобы хранить 2-3 дерево в виде двоичного дерева поиска, заменив наивную реализацию 3-узла на два обычных 2-узла, соединенных ребром особого вида (для возможности отличить "внутреннее ребро" 3-узла от обычного ребра), которое стали называть "красным" (в противоположность "черному", т.е. обычному ребру):

![rbnode](support/rbnode "red and black links")

Как видно, больший (по ключу, т.е. c `KEY_G`) элемент 3-узла с ключами `(KEY_L, KEY_G)` становится родителем, правая ветвь которого суть правая ветвь исходного 3-узла, а левая ветвь есть узел с `KEY_L` и левыми двумя потомками исходного узла (рассматриваем реализацию "левых деревьев", __`left-leaning`__ `red black tree`). 

Эквивалентное определение:
+ Каждый узел имеет не более 1 красной связи;
+ Все пути от `root` до любого `null` содержат одинаковое количество черных связей;
+ Красными могут быть только "левые" ветви.

(Второй пункт поддерживается благодаря постоянству расстояния от `root` до `null` в 2-3-деревьях).



Для поддержания перечисленных инвариантов нам понадобятся некоторые новшества по отношению к обычному двоичного дереву.

Во-первых, т.к. одному узлу всегда соответствует 1 связь от родителя (исключением является только `root`, который не имеет родителя по определению), цвет связи можно отождествить с цветом узла, т.е. сделать его атрибутом класса `Node` (большинство статей, в т.ч. [википедия](https://www.wikiwand.com/ru/%D0%9A%D1%80%D0%B0%D1%81%D0%BD%D0%BE-%D1%87%D1%91%D1%80%D0%BD%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE), сразу говорит о "цвете узлов", пропуская интуицию за этим).

In [2]:
class Node():
    def __init__(self, key=None, value=None):
        self.key = key
        self.value = value
        self.left = None
        self.right = None
        self.count = 1
        self.color = False # Red is `True`, Black is `False` 

Во-вторых, нам понадобятся вспомогательные операции над узлами дерева - два поворота и смена цветов.

+ __Левый поворот (`left rotation`)__: опускаем "налево" данный узел (`root`), его место занимает его правый потомок (`pivot`). Левого потомка `pivot` навешиваем на `root` в качество правого потомка.  
+ __Правый поворот (`right rotation`)__: опускаем "направо" данный узел (`root`), его место занимает его левый потомок (`pivot`). Правого потомка `pivot` навешиваем на `root` в качество левого потомка.
+ __Обращение цветов (`color flip`)__: красим в черный цвет двух красных потомков, сам черный узел `root` красим в красный цвет.

Анимированная картинка с вики:

In [6]:
from IPython.display import Image
Image(url='support/bstrot.gif') 

Важнейшим фактом является то, что все операции (кроме вставки удаления), которые были реализованы для обычного __двоичного дерева поиска__, остаются ровно такими же и для __красно-черного дерева__, только работают быстрее из-за лучшей балансировки. Операции вставки и удаления требуют отдельного внимания, т.к. именно они обеспечивают самобалансировку дерева.

In [7]:
# TODO: реализация put(key, value) и get(key) с использованием поворотов и обращения цветов.